In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000, 28 * 28))
X_train = X_train.astype('float32') / 255
X_test = X_test.reshape((10000, 28 *28))
X_test = X_test.astype('float32') / 255

2024-02-13 16:41:59.663674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 16:41:59.663703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 16:41:59.664569: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 16:41:59.669269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 16:42:00.271679: W tensorflow/compiler/tf2

In [2]:
print('First 3 labels: ', y_train[:3])

First 3 labels:  [5 0 4]


In [3]:
y_train_ohe = tf.keras.utils.to_categorical(y_train)
print('\nFirst 3 labels (one-hot):\n', y_train_ohe[:3])


First 3 labels (one-hot):
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.optimizers.schedules import ExponentialDecay
import numpy as np

np.random.seed(42)

model = Sequential()
model.add(Dense(units=50,
                input_dim=X_train.shape[1],
                kernel_initializer='uniform',
                activation='tanh'))

model.add(Dense(units=50,
                kernel_initializer='uniform',
                activation='tanh'))

model.add(Dense(units=y_train_ohe.shape[1],
                kernel_initializer='uniform',
                activation='softmax'))

initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

sgd = SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

2024-02-13 16:42:01.229555: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 16:42:01.262375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 16:42:01.262563: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
model.fit(X_train,
          y_train_ohe,
          epochs=50,
          batch_size=300,
          verbose=1,
          validation_split=0.1)

Epoch 1/50


2024-02-13 16:42:02.396943: I external/local_xla/xla/service/service.cc:168] XLA service 0x71560f560060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-13 16:42:02.396960: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-02-13 16:42:02.404733: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1707860522.444798  406695 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


180/180 [==============================] - 1s 3ms/step - loss: 2.2983 - accuracy: 0.1827 - val_loss: 2.2930 - val_accuracy: 0.3302
Epoch 2/50
180/180 [==============================] - 0s 2ms/step - loss: 2.2868 - accuracy: 0.3699 - val_loss: 2.2791 - val_accuracy: 0.4077
Epoch 3/50
180/180 [==============================] - 0s 2ms/step - loss: 2.2687 - accuracy: 0.4238 - val_loss: 2.2542 - val_accuracy: 0.4450
Epoch 4/50
180/180 [==============================] - 0s 2ms/step - loss: 2.2313 - accuracy: 0.4357 - val_loss: 2.1976 - val_accuracy: 0.4625
Epoch 5/50
180/180 [==============================] - 0s 2ms/step - loss: 2.1404 - accuracy: 0.4369 - val_loss: 2.0579 - val_accuracy: 0.4508
Epoch 6/50
180/180 [==============================] - 0s 2ms/step - loss: 1.9437 - accuracy: 0.4386 - val_loss: 1.8031 - val_accuracy: 0.4695
Epoch 7/50
180/180 [==============================] - 0s 2ms/step - loss: 1.6750 - accuracy: 0.4856 - val_loss: 1.5173 - val_accuracy: 0.5365
Epoch 8/50
180/18

In [6]:
y_train_pred = np.argmax(model.predict(X_train, verbose=0), axis=1)
print('First 3 predictions: ', y_train_pred[:3])

First 3 predictions:  [3 0 4]


In [7]:
train_acc = np.sum(
    y_train == y_train_pred, axis=0) / X_train.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))

Training accuracy: 92.06%


In [8]:
y_test_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
test_acc = np.sum(
    y_test == y_test_pred, axis=0) / X_test.shape[0]

print('Training accuracy: %.2f%%' % (test_acc * 100))

Training accuracy: 92.04%
